In [392]:
#必要なライブラリをインポート
import pandas as pd
import numpy as np
from IPython.display import display

df = pd.read_csv('csv/chiyoda_chuo_taito.csv', sep='\t', encoding='UTF-8')

In [393]:
# 不要な列を削除
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [394]:
# df[df['間取り'] in ['1LDK', '2DK', '2LDK', '2SLDK', '3K', '3DK', '3LDK', '3SLDK']] がエラー出てびびった
df[df['間取り'].isin(['1LDK', '2DK', '2LDK', '2SLDK', '3K', '3DK', '3LDK', '3SLDK'])]

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積,URL
0,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,13階,20.5万円,-,20.5万円/20.5万円/-/-,1LDK,42.04m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
1,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,30階,22万円,-,44万円/22万円/-/-,1LDK,58.25m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
2,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,39階,24万円,-,24万円/24万円/-/-,1LDK,49.95m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
4,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,22階,24.5万円,20000円,24.5万円/24.5万円/-/-,2LDK,58.94m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
5,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,20階,26万円,20000円,26万円/26万円/-/-,2LDK,68.62m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
6,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,14階,28.5万円,-,57万円/28.5万円/-/-,3LDK,76.14m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
7,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,44階,27.5万円,15000円,27.5万円/27.5万円/-/-,3LDK,78.02m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
8,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,14階,28.5万円,15000円,57万円/28.5万円/-/-,3LDK,76.14m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
9,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,27階,28万円,20000円,28万円/28万円/-/-,2LDK,67.86m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361
11,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,築1年,10階建,33階,37万円,-,74万円/37万円/-/-,2LDK,79.84m,https://suumo.jp/chintai/jnc_000041057199/?bc=100124119361


In [ ]:
# 立地を「路線+駅」と「徒歩〜分」に分割
splitted1 = df['立地1'].str.split(' 歩', expand=True)
splitted1.columns = ['立地11', '立地12']
splitted2 = df['立地2'].str.split(' 歩', expand=True)
splitted2.columns = ['立地21', '立地22']
splitted3 = df['立地3'].str.split(' 歩', expand=True)
splitted3.columns = ['立地31', '立地32']

In [396]:
#その他費用を、「敷金」「礼金」「保証金」「敷引,償却」に分割
splitted4 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
splitted4.columns = ['敷金', '礼金', '保証金', '敷引,償却']

In [409]:
df = pd.concat([df, splitted1, splitted2, splitted3, splitted4], axis=1)
df.head()

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,...,立地11,立地12,立地21,立地22,立地31,立地32,敷金,礼金,保証金,"敷引,償却"
0,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,13階,205000.0,0.0,...,都営大江戸線/勝どき駅,6分,東京メトロ有楽町線/月島駅,12分,東京メトロ日比谷線/築地駅,16分,20.5万円,20.5万円,-,-
1,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,30階,220000.0,0.0,...,都営大江戸線/勝どき駅,6分,東京メトロ有楽町線/月島駅,12分,東京メトロ日比谷線/築地駅,16分,44万円,22万円,-,-
2,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,39階,240000.0,0.0,...,都営大江戸線/勝どき駅,6分,東京メトロ有楽町線/月島駅,12分,東京メトロ日比谷線/築地駅,16分,24万円,24万円,-,-
3,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,39階,238000.0,14000.0,...,都営大江戸線/勝どき駅,6分,東京メトロ有楽町線/月島駅,12分,東京メトロ日比谷線/築地駅,16分,23.8万円,23.8万円,-,-
4,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,22階,245000.0,20000.0,...,都営大江戸線/勝どき駅,6分,東京メトロ有楽町線/月島駅,12分,東京メトロ日比谷線/築地駅,16分,24.5万円,24.5万円,-,-


In [398]:
#数値として扱いたいので、不要な文字列を削除
df['賃料'] = df['賃料'].str.replace(u'万円', u'')
df['敷金'] = df['敷金'].str.replace(u'万円', u'')
df['礼金'] = df['礼金'].str.replace(u'万円', u'')
df['保証金'] = df['保証金'].str.replace(u'万円', u'')
df['管理費'] = df['管理費'].str.replace(u'円', u'')
df['築年数'] = df['築年数'].str.replace(u'新築', u'0') #新築は築年数0年とする
df['築年数'] = df['築年数'].str.replace(u'築', u'')
df['築年数'] = df['築年数'].str.replace(u'年', u'')
df['専有面積'] = df['専有面積'].str.replace(u'm', u'')
df['立地12'] = df['立地12'].str.replace(u'分', u'')
df['立地22'] = df['立地22'].str.replace(u'分', u'')
df['立地32'] = df['立地32'].str.replace(u'分', u'')

In [399]:
#「-」を0に変換
df['管理費'] = df['管理費'].replace('-',0)
df['敷金'] = df['敷金'].replace('-',0)
df['礼金'] = df['礼金'].replace('-',0)
df['保証金'] = df['保証金'].replace('-',0)

In [400]:
#文字列から数値に変換
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['保証金'] = pd.to_numeric(df['保証金'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])
df['立地12'] = pd.to_numeric(df['立地12'])
df['立地22'] = pd.to_numeric(df['立地22'])
df['立地32'] = pd.to_numeric(df['立地32'])

In [401]:
#単位を合わせるために、管理費以外を10000倍。
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000
df['保証金'] = df['保証金'] * 10000

In [402]:
#管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']

#敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['敷/礼/保証金'] = df['敷金'] + df['礼金'] + df['保証金']

In [403]:
df

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,...,立地21,立地22,立地31,立地32,敷金,礼金,保証金,"敷引,償却",賃料+管理費,敷/礼/保証金
0,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,13階,205000.0,0.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,205000.0,205000.0,0.0,-,205000.0,410000.0
1,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,30階,220000.0,0.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,440000.0,220000.0,0.0,-,220000.0,660000.0
2,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,39階,240000.0,0.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,240000.0,240000.0,0.0,-,240000.0,480000.0
3,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,39階,238000.0,14000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,238000.0,238000.0,0.0,-,252000.0,476000.0
4,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,22階,245000.0,20000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,245000.0,245000.0,0.0,-,265000.0,490000.0
5,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,20階,260000.0,20000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,260000.0,260000.0,0.0,-,280000.0,520000.0
6,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,14階,285000.0,0.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,570000.0,285000.0,0.0,-,285000.0,855000.0
7,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,44階,275000.0,15000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,275000.0,275000.0,0.0,-,290000.0,550000.0
8,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,14階,285000.0,15000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,570000.0,285000.0,0.0,-,300000.0,855000.0
9,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10階建,27階,280000.0,20000.0,...,東京メトロ有楽町線/月島駅,12.0,東京メトロ日比谷線/築地駅,16.0,280000.0,280000.0,0.0,-,300000.0,560000.0


In [404]:
#住所を「東京都」「〜区」「市町村番地」に分割
splitted6 = df['住所'].str.split('区', expand=True)
splitted6.columns = ['区', '市町村']
splitted6['区'] = splitted6['区'] + '区'
splitted6['区'] = splitted6['区'].str.replace('東京都','')

In [405]:
#階を数値化。地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'].str.encode('cp932')
splitted10['階1'] = splitted10['階1'].str.replace(u'階', u'')
splitted10['階1'] = splitted10['階1'].str.replace(u'B', u'-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])
df = pd.concat([df, splitted10], axis=1)

0        b'13\x8aK'
1        b'30\x8aK'
2        b'39\x8aK'
3        b'39\x8aK'
4        b'22\x8aK'
5        b'20\x8aK'
6        b'14\x8aK'
7        b'44\x8aK'
8        b'14\x8aK'
9        b'27\x8aK'
10       b'14\x8aK'
11       b'33\x8aK'
12       b'45\x8aK'
13       b'53\x8aK'
14        b'2\x8aK'
15        b'2\x8aK'
16       b'10\x8aK'
17        b'5\x8aK'
18        b'5\x8aK'
19       b'13\x8aK'
20       b'26\x8aK'
21        b'6\x8aK'
22        b'6\x8aK'
23        b'9\x8aK'
24        b'5\x8aK'
25        b'7\x8aK'
26        b'9\x8aK'
27       b'10\x8aK'
28        b'3\x8aK'
29        b'8\x8aK'
            ...    
10634     b'7\x8aK'
10635     b'8\x8aK'
10636     b'6\x8aK'
10637     b'6\x8aK'
10638     b'2\x8aK'
10639     b'1\x8aK'
10640     b'1\x8aK'
10641     b'1\x8aK'
10642     b'4\x8aK'
10643     b'3\x8aK'
10644     b'5\x8aK'
10645     b'3\x8aK'
10646     b'2\x8aK'
10647     b'4\x8aK'
10648     b'3\x8aK'
10649     b'5\x8aK'
10650     b'5\x8aK'
10651     b'2\x8aK'
10652     b'2\x8aK'


In [406]:
#建物高さを数値化。地下は無視。
df['建物高さ'].str.encode('cp932')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下1地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下2地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下3地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下4地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下5地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下6地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下7地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下8地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下9地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'平屋', u'1')
df['建物高さ'] = df['建物高さ'].str.replace(u'階建', u'')
df['建物高さ'] = pd.to_numeric(df['建物高さ'])

0        b'10\x8aK\x8c\x9a'
1        b'10\x8aK\x8c\x9a'
2        b'10\x8aK\x8c\x9a'
3        b'10\x8aK\x8c\x9a'
4        b'10\x8aK\x8c\x9a'
5        b'10\x8aK\x8c\x9a'
6        b'10\x8aK\x8c\x9a'
7        b'10\x8aK\x8c\x9a'
8        b'10\x8aK\x8c\x9a'
9        b'10\x8aK\x8c\x9a'
10       b'10\x8aK\x8c\x9a'
11       b'10\x8aK\x8c\x9a'
12       b'10\x8aK\x8c\x9a'
13       b'10\x8aK\x8c\x9a'
14       b'10\x8aK\x8c\x9a'
15       b'10\x8aK\x8c\x9a'
16       b'10\x8aK\x8c\x9a'
17       b'10\x8aK\x8c\x9a'
18       b'10\x8aK\x8c\x9a'
19       b'10\x8aK\x8c\x9a'
20       b'10\x8aK\x8c\x9a'
21       b'10\x8aK\x8c\x9a'
22       b'10\x8aK\x8c\x9a'
23       b'10\x8aK\x8c\x9a'
24       b'10\x8aK\x8c\x9a'
25       b'10\x8aK\x8c\x9a'
26       b'10\x8aK\x8c\x9a'
27       b'10\x8aK\x8c\x9a'
28       b'10\x8aK\x8c\x9a'
29       b'10\x8aK\x8c\x9a'
                ...        
10634                   NaN
10635                   NaN
10636                   NaN
10637                   NaN
10638               

In [407]:
df

,マンション名,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,...,立地31,立地32,敷金,礼金,保証金,"敷引,償却",賃料+管理費,敷/礼/保証金,階1,階2
0,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,13階,205000.0,0.0,...,東京メトロ日比谷線/築地駅,16.0,205000.0,205000.0,0.0,-,205000.0,410000.0,13.0,None
1,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,30階,220000.0,0.0,...,東京メトロ日比谷線/築地駅,16.0,440000.0,220000.0,0.0,-,220000.0,660000.0,30.0,None
2,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,39階,240000.0,0.0,...,東京メトロ日比谷線/築地駅,16.0,240000.0,240000.0,0.0,-,240000.0,480000.0,39.0,None
3,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,39階,238000.0,14000.0,...,東京メトロ日比谷線/築地駅,16.0,238000.0,238000.0,0.0,-,252000.0,476000.0,39.0,None
4,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,22階,245000.0,20000.0,...,東京メトロ日比谷線/築地駅,16.0,245000.0,245000.0,0.0,-,265000.0,490000.0,22.0,None
5,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,20階,260000.0,20000.0,...,東京メトロ日比谷線/築地駅,16.0,260000.0,260000.0,0.0,-,280000.0,520000.0,20.0,None
6,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,14階,285000.0,0.0,...,東京メトロ日比谷線/築地駅,16.0,570000.0,285000.0,0.0,-,285000.0,855000.0,14.0,None
7,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,44階,275000.0,15000.0,...,東京メトロ日比谷線/築地駅,16.0,275000.0,275000.0,0.0,-,290000.0,550000.0,44.0,None
8,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,14階,285000.0,15000.0,...,東京メトロ日比谷線/築地駅,16.0,570000.0,285000.0,0.0,-,300000.0,855000.0,14.0,None
9,勝どきザ・タワー,東京都中央区勝どき５,都営大江戸線/勝どき駅 歩6分,東京メトロ有楽町線/月島駅 歩12分,東京メトロ日比谷線/築地駅 歩16分,1.0,10.0,27階,280000.0,20000.0,...,東京メトロ日比谷線/築地駅,16.0,280000.0,280000.0,0.0,-,300000.0,560000.0,27.0,None


In [408]:
#カラムを入れ替えて、csvファイルとして出力
df[['マンション名','住所','間取り','築年数','建物高さ','階1','専有面積','賃料+管理費','敷/礼/保証金',
                '賃料','管理費','敷金','礼金','保証金']]

df.to_csv('all.csv', sep = '\t',encoding='UTF-8')

,マンション名,住所,間取り,築年数,建物高さ,階1,専有面積,賃料+管理費,敷/礼/保証金,賃料,管理費,敷金,礼金,保証金
0,勝どきザ・タワー,東京都中央区勝どき５,1LDK,1.0,10.0,13.0,42.04,205000.0,410000.0,205000.0,0.0,205000.0,205000.0,0.0
1,勝どきザ・タワー,東京都中央区勝どき５,1LDK,1.0,10.0,30.0,58.25,220000.0,660000.0,220000.0,0.0,440000.0,220000.0,0.0
2,勝どきザ・タワー,東京都中央区勝どき５,1LDK,1.0,10.0,39.0,49.95,240000.0,480000.0,240000.0,0.0,240000.0,240000.0,0.0
3,勝どきザ・タワー,東京都中央区勝どき５,1SLDK,1.0,10.0,39.0,58.94,252000.0,476000.0,238000.0,14000.0,238000.0,238000.0,0.0
4,勝どきザ・タワー,東京都中央区勝どき５,2LDK,1.0,10.0,22.0,58.94,265000.0,490000.0,245000.0,20000.0,245000.0,245000.0,0.0
5,勝どきザ・タワー,東京都中央区勝どき５,2LDK,1.0,10.0,20.0,68.62,280000.0,520000.0,260000.0,20000.0,260000.0,260000.0,0.0
6,勝どきザ・タワー,東京都中央区勝どき５,3LDK,1.0,10.0,14.0,76.14,285000.0,855000.0,285000.0,0.0,570000.0,285000.0,0.0
7,勝どきザ・タワー,東京都中央区勝どき５,3LDK,1.0,10.0,44.0,78.02,290000.0,550000.0,275000.0,15000.0,275000.0,275000.0,0.0
8,勝どきザ・タワー,東京都中央区勝どき５,3LDK,1.0,10.0,14.0,76.14,300000.0,855000.0,285000.0,15000.0,570000.0,285000.0,0.0
9,勝どきザ・タワー,東京都中央区勝どき５,2LDK,1.0,10.0,27.0,67.86,300000.0,560000.0,280000.0,20000.0,280000.0,280000.0,0.0
